In [1]:
import telegram
from telegram.ext import Updater
import logging
from telegram.ext import CommandHandler, MessageHandler, Filters
import io, requests

In [2]:
import numpy as np
np.random.seed(42)
import pandas as pd
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
import cv2

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
bot = telegram.Bot(token='TOKEN')
print(bot.get_me())

{'id': 929294392, 'first_name': 'Face Bot', 'is_bot': True, 'username': 'FaceClassificationBot'}


In [4]:
### Define updater object
updater = Updater(token='TOKEN', use_context=True)
dispatcher = updater.dispatcher ### Defines a dispatcher for messages
updater.stop()   

In [5]:
### Logging to keep track of pings
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [6]:
### define a start function for the bot to message
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text="I'm the face classification bot, please send me a picture of yourself!")

In [7]:
### tells the bot when to run the start function
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [8]:
### stop previous instances, run the bot!
updater.start_polling() 

In [9]:
def get_image_id_from_message(message):
    return message.photo[len(message.photo)-1].file_id

In [10]:
def classifyFace(cid,message):
    bot.send_message(cid, '🔥 Analysing image, please wait 🔥')
    imageId = get_image_id_from_message(message)
    url = bot.get_file(imageId).file_path
    imgStream = io.BytesIO(requests.get(url).content)
    photo = cv2.imdecode(np.fromstring(imgStream.read(), np.uint8), 1)
    print(photo.shape)
    detected = False
    results = MTCNN().detect_faces(photo)
    if len(results)!=0:
        detected= True
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = photo[y1:y2, x1:x2]
        face = cv2.resize(face,(150,150))
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        x = (face-face.mean())/face.std()
        x = np.array(x,dtype=np.float32)
        x = x.reshape(1,150,150,1)
        modelEmotion = load_model('emotionBest_0.760_0.728.h5')
        resultEmotion = (modelEmotion.predict(x)).tolist()[0]
        emotionsDict = {0:'Happy', 1:'Neutral', 2:'Sad/Angry', 3:'Surprised/Fearful'}
        emotion = emotionsDict[resultEmotion.index(max(resultEmotion))]
        modelGender = load_model('cnnModelGender1.h5')
        resultGender = int(round(modelGender.predict(x).tolist()[0][0]))
        genderDict = {0:'Female', 1:'Male'}
        gender = genderDict[resultGender]
        modelAge = load_model('cnnModelAge1.h5')
        age = int(round(modelAge.predict(x).tolist()[0][0]))
        return str(emotion), str(gender), str(age)
    else:
        return 'Sorry, no face detected'

In [11]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text=classifyFace(update.effective_chat.id,update.message))
echo_handler = MessageHandler(Filters.photo, echo)
dispatcher.add_handler(echo_handler)

In [12]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
dt_string

'28/01/2020 10:39:25'

In [13]:
def info(update, context):
    info_text = "I'm a telegram bot! I was last updated at " + dt_string + "."
    context.bot.send_message(chat_id=update.effective_chat.id, text= info_text)
caps_handler = CommandHandler('info', info)
dispatcher.add_handler(caps_handler)

In [14]:
def unknown(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I didn't understand that command.")

unknown_handler = MessageHandler(Filters.command, unknown)
dispatcher.add_handler(unknown_handler)

In [15]:
def caps(update, context):
    text_caps = ' '.join(context.args).upper()
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_caps)

caps_handler = CommandHandler('caps', caps)
dispatcher.add_handler(caps_handler)

(600, 800, 3)



C:\Users\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':
2020-01-28 10:39:39,754 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-01-28 10:39:39,797 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-01-28 10:39:39,813 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2020-01-28 10:39:39,860 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



...

2020-01-28 10:39:41,429 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-01-28 10:39:41,854 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-01-28 10:39:41,980 - tensorflow - WARNING - From C:\Users\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
C:\Users\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':


(600, 800, 3)
